# RAG using Llama-3

### Loading libraries and dependencies

In [17]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama

### Pointing the data path

In [18]:
input_dir_path = './data'

### Loading embedding and LLM model

Check more here:

- [Hugging Face Transformers](https://huggingface.co/models)
- [Ollama ](https://ollama.io/)

In [19]:
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

Settings.llm = Ollama(model="llama3.2", request_timeout=360.0)

### Loading the data

In [20]:
documents = SimpleDirectoryReader(
                input_dir=input_dir_path,
                required_exts=[".pdf"],
                recursive=True
            ).load_data()

### Data embedding

In [21]:
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine()

### Query and response

In [22]:
response = query_engine.query("Hàng hóa của tôi bị thất lạc thì bồi thường bao nhiêu?")
print(response)

Giá trị bồi thường cho hàng hóa thất lạc là 4 lần cước phí.


### Adding some custom prompt

In [23]:
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information above I want you to think step by step to answer the query in a crisp manner, "
    "incase you don't know the answer say 'I don't know!'.\n"
    "Query: {query_str}\n"
    "Answer: "
)

qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

### Query and response

In [24]:
response = query_engine.query("Thời gian giao hàng là bao lâu?")
print(response)

Tôi sẽ phân tích thông tin từ context information:

* Có hai thời gian lấy hàng được chỉ định:
 + Lần 1 (10h30 - 12h00): Đơn hàng đầu tiên tiềm đến sẽ được lấy hàng.
 + Lần 2 (16h00 - 17h30): Đơn hàng đầu tiên tiềm đến sẽ được lấy hàng.

Tuy nhiên, không có thông tin cụ thể về thời gian giao hàng. Nhưng chúng ta có thể suy ra rằng:

* Thời gian giao hàng là bao lâu?
Tôi không biết chính xác thời gian giao hàng từ văn bản trên.
